In [1]:
# load the environment variables
from dotenv import load_dotenv
from pathlib import Path

# add an environment file with the secrets if needed
# and make sure to exclude it from git!!
env_path = Path('.') / 'env-secrets.txt'
load_dotenv(dotenv_path=env_path)

True

In [2]:
import os
openai_api_key = os.getenv("OPENAI_SOILWISE_API_KEY")

In [3]:
from langchain.chains import GraphSparqlQAChain
from langchain_community.graphs import RdfGraph
from langchain_openai import ChatOpenAI

In [29]:
g = RdfGraph(
    source_file="http://www.w3.org/People/Berners-Lee/card",
    standard="rdf",
    local_copy="test.ttl",
)

In [30]:
# from rdflib import Graph

# Create a Graph, pare in Internet data
# g = Graph().parse("http://www.w3.org/People/Berners-Lee/card")

# Query the data in g using SPARQL
# This query returns the 'name' of all ``foaf:Person`` instances
q = """
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?name
    WHERE {
        ?p rdf:type foaf:Person .

        ?p foaf:name ?name .
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["name"])

Timothy Berners-Lee


In [31]:
print(g.get_schema)

In the following, each IRI is followed by the local name and optionally its description in parentheses. 
The RDF graph supports the following node types:
<http://xmlns.com/foaf/0.1/PersonalProfileDocument> (PersonalProfileDocument, None), <http://www.w3.org/ns/auth/cert#RSAPublicKey> (RSAPublicKey, None), <http://www.w3.org/2000/10/swap/pim/contact#Male> (Male, None), <http://xmlns.com/foaf/0.1/Person> (Person, None), <http://www.w3.org/2006/vcard/ns#Work> (Work, None)
The RDF graph supports the following relationships:
<http://www.w3.org/2003/01/geo/wgs84_pos#lat> (lat, None), <http://xmlns.com/foaf/0.1/openid> (openid, None), <http://purl.org/dc/elements/1.1/title> (title, None), <http://www.w3.org/ns/solid/terms#profileBackgroundColor> (profileBackgroundColor, None), <http://xmlns.com/foaf/0.1/homepage> (homepage, None), <http://www.w3.org/2000/10/swap/pim/contact#assistant> (assistant, None), <http://www.w3.org/ns/solid/terms#publicTypeIndex> (publicTypeIndex, None), <http://www.w3

In [32]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
chain = GraphSparqlQAChain.from_llm(llm=llm, graph=g, return_sparql_query=False, verbose=True)

In [35]:
GraphSparqlQAChain?

In [33]:
result = chain.invoke("""
    What is Tim Berners-Lee work homepage?
    
    When querying use only the family_name property from the graph.
    Return both the person and the homepage URL.
    Make sure that the created SPARQL query is UTF-8 compliant.
""")

print(f"SPARQL query: {result['sparql_query']}")
# print(f"Final answer: {result['result']}")



> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
```
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?person ?homepage
WHERE {
    ?person foaf:family_name "Berners-Lee" .
    ?person foaf:workplaceHomepage ?homepage .
}
```


ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '`'  (at char 0), (line:1, col:1)

In [34]:
query = '''
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?person ?homepage
WHERE {
    ?person foaf:family_name "Berners-Lee" .
    ?person foaf:workplaceHomepage ?homepage .
}
'''

g.query(query)

[(rdflib.term.URIRef('https://www.w3.org/People/Berners-Lee/card#i'),
  rdflib.term.URIRef('https://www.w3.org/'))]